In [2]:
import os
import sys
sys.path.append(os.path.join('..'))

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
%load_ext autoreload
%autoreload 2

from src.content import content

## Init the model. We will start with the api solution

In [4]:
openai_model = "gpt-3.5-turbo" # "gpt-4o" "gpt-3.5-turbo-1106", "gpt-3.5-turbo-0125"

# Count tokens with Tiktoken
https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken

In [5]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(content, openai_model)

2309

# Generate a single role-play card.

1. Set a project name.
2. They were suggesting me a stream there, but i am not sure if i can use it in telegram. <br> https://cookbook.openai.com/examples/how_to_stream_completions

In [6]:
def format_json_pretty(json_data):
    formatted_str = ""
    for key, value in json_data.items():
        if isinstance(value, dict):
            formatted_str += f"**{key}**\n"
            for sub_key, sub_value in value.items():
                formatted_str += f"  {sub_key}. {sub_value}\n"
        elif isinstance(value, list):
            formatted_str += f"**{key}**\n"
            for sub_value in value:
                formatted_str += f"  - {sub_value}\n"
        else:
            formatted_str += f"**{key}**: {value}\n"
    return formatted_str

In [7]:
import json
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv('OPENAI_API_OET'),
    organization=os.getenv('OPENAI_ORGANIZATION_ID'), # unnecessary
    project=os.getenv('OPENAI_PROJ_OET'), # to track expenses
)

In [8]:
%%time

completion = client.chat.completions.create(
    model=openai_model,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": content},
        {"role": "user", "content": "Create an OET card in JSON format"}
    ],
    # stream=True, # don't need for card generation
    max_tokens=1000
)

card_content = completion.choices[0].message.content
card_json = json.loads(card_content)

# print(completion.model, completion.usage.to_dict())

assert len(card_json.items()) == 2, 'The card should consist of two parts'

cards = {}
for card_name, (key, val) in zip(['patent', 'doctor'], card_json.items()):
    print(key, format_json_pretty(val), sep=':\n')
    cards[card_name] = (key + ':\n' + format_json_pretty(val))

Patient Card:
**SETTING**: Local Hospital Emergency Department
**PATIENT**: You are a 30-year-old female who recently returned from a trip to South America. You have been experiencing high fever, severe headache, muscle pain, and fatigue for the past week. You suspect you may have contracted a tropical disease like dengue fever or malaria.
**PATIENT TASKS**
  - When asked, mention that you have had chills and night sweats during this time.
  - Express concern about the possibility of having a serious tropical disease.
  - Inform the doctor that you did not take any antimalarial medication before your trip.
  - Ask about the symptoms and differences between dengue fever and malaria.
  - Agree to undergo any necessary tests or treatments recommended by the doctor.

Doctor Card:
**SETTING**: Local Hospital Emergency Department
**DOCTOR**: You are seeing a 30-year-old female patient who recently returned from a trip to South America and is presenting with symptoms of high fever, severe hea

In [9]:
completion.model

'gpt-3.5-turbo-0125'

In [8]:
print(cards['doctor'])

Doctor Card:
**SETTING**: Local Pharmacy
**DOCTOR**: You encounter a 30-year-old female patient at the pharmacy who has been suffering from persistent headaches for a week and seeks advice on managing them.
**DOCTOR TASK**
  - Inquire about the nature of the headaches (location, intensity, accompanying symptoms like nausea and sensitivity to light).
  - Ask about the patient's daily habits and work environment to identify possible triggers (e.g., prolonged screen time, stress).
  - Discuss the potential causes of the headaches (e.g., tension headaches, migraines) and reassure the patient about different treatment options available.
  - Recommend lifestyle modifications such as taking regular breaks from screen time, staying hydrated, and practicing relaxation techniques to help manage the headaches.
  - Advise the patient on over-the-counter pain relief options and when to consider seeking further medical evaluation if the headaches persist or worsen.



# Role play game

## TODO:
    1 Придумать способ учитывать уже сделанные карточки, чтобы пользователь видел разные.
    2 Придумать способ регенерации карточки для role-play если вышло говно. 
        * Один раз уже было такое, что не было 2-х объектов в словаре на выходе. 

# 1. Creating and Sending the Card to the User

Let's start by creating a Role-Play card.

In [9]:
def generate_card():
    completion = client.chat.completions.create(
        model=openai_model,
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": "Create an OET card in JSON format"}
        ],
        # stream=True, # don't need for card generation
        max_tokens=1000
    )
    
    card_content = completion.choices[0].message.content
    card_json = json.loads(card_content)
    
    # print(completion.model, completion.usage.to_dict())
    
    assert len(card_json.items()) == 2, 'The card should consist of two parts'

    cards = {}
    for card_name, (key, val) in zip(['patient', 'doctor'], card_json.items()):
        cards[card_name] = (key + ':\n' + format_json_pretty(val))
    
    return cards, card_json

formatted_cards, card_json = generate_card()
print(formatted_cards['patient'], formatted_cards['doctor'], sep='\n')

Patient Card:
**SETTING**: Dermatology Clinic
**PATIENT**: You are a 30-year-old woman who has been experiencing severe itching and redness on your skin for the past few weeks. You have noticed the symptoms mainly on your arms, legs, and back. The itching is disrupting your sleep and daily activities, and you have tried over-the-counter creams with no relief.
**PATIENT TASKS**
  - Describe the severity of the itching and redness on your skin, mentioning the areas affected.
  - Express how the itching is impacting your sleep and daily life.
  - Inform the doctor about the unsuccessful use of over-the-counter creams.
  - Ask the doctor about the possible causes of your symptoms and if it could be a skin allergy.
  - Acknowledge the doctor's recommendations and ask about the next steps.

Doctor Card:
**SETTING**: Dermatology Clinic
**DOCTOR**: Your patient is a 30-year-old woman presenting with severe itching and redness on her arms, legs, and back. She reports disrupted sleep and daily a

# 2. Show the Card and wait 3 minutes.
* At this point we have to show the user his card and let him prepare with it for 3 minutes

In [10]:
import time


print('You know have 3 minutes to prepare', formatted_cards['doctor'], sep='\n')
time.sleep(1) ## 180

You know have 3 minutes to prepare
Doctor Card:
**SETTING**: Dermatology Clinic
**DOCTOR**: Your patient is a 30-year-old woman presenting with severe itching and redness on her arms, legs, and back. She reports disrupted sleep and daily activities despite trying over-the-counter creams.
**DOCTOR TASKS**
  - Inquire about the duration and progression of the symptoms.
  - Discuss potential triggers for the itching and redness (e.g., new products, environmental changes).
  - Explain the possible causes of the symptoms (e.g., contact dermatitis, eczema) and the importance of a proper diagnosis.
  - Recommend allergy testing or patch testing to identify potential allergens.
  - Provide treatment options such as prescription creams, lifestyle modifications, and follow-up care.



# 3. Generate a Voice Message.
* Voice Message from the User. It will be just sent to us in telegram, but here I need to generate it.

## 3.1 Google Text-to-Speech 
* I liked this the most

In [11]:
# !pip install gtts -q

In [15]:
from gtts import gTTS

user = 'Pete'
text = f"Hello, my name is {user}, I am a doctor of this clinic. May I know your name and please tell me what is bothering you?"

tts = gTTS(text=text, lang='en')

path = f"../data/audio_storage/{user}_intro_gtts.mp3"
tts.save(path)

os.system(f"start {path}")  # For Windows

0

In [16]:
def create_audio(text, path):
    tts = gTTS(text=text, lang='en')
    tts.save(path)

## 3.2 pyttsx3 is a text-to-speech conversion library in Python

* Didn't like the sound.

In [17]:
# import pyttsx3

# # Initialize the TTS engine
# engine = pyttsx3.init()

# # Text to convert to speech
# text = f"Hello, my name is {user}, I am a doctor of this clinic. May I know your name and please tell me what is bothering you?"

# # Set properties (optional)
# engine.setProperty('rate', 200)  # Speed of speech
# engine.setProperty('volume', 0.9)  # Volume (0.0 to 1.0)

# # Convert text to speech
# path = f"data/audio_storage/{user}_intro_pyttsx3.mp3"
# engine.save_to_file(text, path)
# engine.runAndWait()

# os.system(f"start {path}")  # For Windows

## 3.3 TTS

In [18]:
# import torch
# from TTS.api import TTS

# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(device)

# # print(TTS().list_models().list_tts_models())

# tts = TTS('tts_models/en/ljspeech/speedy-speech').to(device)

# text = f"Hello, my name is {user}, I am a doctor of this clinic. May I know your name and please tell me what is bothering you?"

# path = f"data/audio_storage/{user}_intro_tts.mp3"
# tts.tts_to_file(text=text, file_path=path) # speaker_wav="my/cloning/audio.wav"

# os.system(f"start {path}") # For Windows

# 4. Transcribing the Voice Message with Whisper

In [19]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [20]:
# !pip install git+https://github.com/openai/whisper.git

In [21]:
import torch
import torchaudio
from torch.cuda.amp import autocast
import whisper

device = "cuda" if torch.cuda.is_available() else "CPU"
print(device)

# Select from the following models: "tiny", "base", "small", "medium", "large"
whisper_model = whisper.load_model("base", device=device)

cuda


In [22]:
def load_audio(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    return waveform, sample_rate

def create_audio_chunks(waveform, total_duration, sample_rate, chunk_duration=30):
    chunks = []
    for start in range(0, int(total_duration), chunk_duration):
        end = min(start + chunk_duration, total_duration)
        chunk_waveform = waveform[:, int(start * sample_rate):int(end * sample_rate)]
        chunks.append(chunk_waveform)
    return chunks, sample_rate

def transcribe_with_whisper(model, audio_path):
    if not os.path.isfile(audio_path):
        raise FileNotFoundError(f"The file {audio_path} does not exist.")
    else: 
        waveform, sample_rate = load_audio(audio_path)
        total_duration = waveform.size(1) / sample_rate
        
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is not available. Please check your CUDA installation.")

    if total_duration <= 30:
        waveform = waveform.to(device)

        # Whisper expects audio to be resampled to 16000 Hz
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000).to(device)
        waveform = resampler(waveform)

        audio_tensor = waveform.squeeze(0).cpu().numpy()  # Whisper expects a 1D numpy array
        options = {"fp16": False, "language": None, "task": "transcribe"}
        result = model.transcribe(audio_tensor, **options)
        
        return result['text']
    else:
        chunks, sample_rate = create_audio_chunks(waveform, total_duration, sample_rate)
    
        # Whisper expects audio to be resampled to 16000 Hz
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000).to(device)
        transcriptions = []
    
        for chunk in chunks:
            chunk = chunk.to(device)
            chunk = resampler(chunk)
            
            with autocast():
                audio_tensor = chunk.squeeze(0).cpu().numpy()  # Whisper expects a 1D numpy array
                result = model.transcribe(audio_tensor)
                transcriptions.append(result['text'])
    
            torch.cuda.empty_cache() # Release GPU memory
    
        return " ".join(transcriptions)

## 4.1 Small text transcribing

In [23]:
%%time

transcribed_text = transcribe_with_whisper(whisper_model, path)
transcribed_text

CPU times: total: 3.22 s
Wall time: 1.69 s


' Hello, my name is Pete. I am a doctor of this clinic. May I know your name and please tell me what is bothering you?'

## 4.2 Big text transcribing

In [24]:
long_text = '''
In a bustling town, there lived a curious kitten named Milo. Milo had bright green eyes and a fluffy tail that always seemed to be in motion. He loved to explore every nook and cranny, making new friends along the way. One day, while chasing a butterfly, Milo wandered farther than ever before and stumbled upon an old, enchanted garden.
In the garden, flowers sang melodies, and trees whispered ancient secrets. Milo's whiskers twitched with excitement. As he ventured deeper, he found a tiny, golden door at the base of a giant oak tree. Unable to resist, Milo pushed it open and stepped inside.
To his amazement, he found himself in a land of miniature creatures. Tiny fairies flitted about, and little gnomes tended to their mushroom houses. The ruler of this land, Queen Lila, approached Milo with a warm smile. "Welcome, Milo. We need your help," she said.
Queen Lila explained that a mischievous squirrel named Nibbles had stolen their magical acorn, which kept the garden alive. Without it, the garden would wither away. Brave and determined, Milo agreed to help.
With the fairies' guidance, Milo tracked down Nibbles and persuaded him to return the acorn. In gratitude, Queen Lila gifted Milo a magical collar that would always guide him home. Milo returned to his town, carrying memories of a fantastical adventure and a heart full of new friendships. From then on, Milo knew that no matter where his curiosity led him, he would always find his way back home.
'''

In [36]:
%%time

user = 'Pete'
long_audio_path = f"../data/audio_storage/{user}_long_gtts.mp3"
create_audio(long_text, long_audio_path)

CPU times: total: 1.09 s
Wall time: 8.85 s


In [37]:
%%time

transcribed_text = transcribe_with_whisper(whisper_model, long_audio_path)
transcribed_text

CPU times: total: 12.2 s
Wall time: 6.36 s


' in a bustling town. Their lived a curious kitten named Milo. Milo had bright green eyes and a fluffy tail that always seemed to be in motion. He loved to explore every nook and cranny, making new friends along the way. One day, while chasing a butterfly, Milo wandered farther than ever before and stumbled upon an old, enchanted garden. In the garden, flowers sang melodies.  and Trees whispered ancient secrets. Milo\'s whiskers twitched with excitement. As he ventured deeper, he found a tiny, golden door at the base of a giant oak tree, unable to resist. Milo pushed it open and stepped inside. To his amazement, he found himself in a land of miniature creatures. Tiny fairies flittered about. And little gnomes tended to their mushroom houses. The ruler...  of this land. Queen Lila approached Milo with a warm smile. Welcome! Milo! We need your help!" she said. Queen Lila explained that a mischievous squirrel named nibbles had stolen their magical acorn. Which kept the garden alive. Witho

# 4* Assessing the accuracy of text transcription

In [38]:
import re
import Levenshtein
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import ndiff

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def calculate_levenshtein_distance(original_text, transcribed_text):
    original_text_norm = normalize_text(original_text)
    transcribed_text_norm = normalize_text(transcribed_text)
    distance = Levenshtein.distance(original_text_norm, transcribed_text_norm)
    return distance

def calculate_cosine_similarity(original_text, transcribed_text):
    original_text_norm = normalize_text(original_text)
    transcribed_text_norm = normalize_text(transcribed_text)

    original_tokens = word_tokenize(original_text_norm)
    transcribed_tokens = word_tokenize(transcribed_text_norm)

    original_text_combined = ' '.join(original_tokens)
    transcribed_text_combined = ' '.join(transcribed_tokens)

    vectorizer = TfidfVectorizer().fit_transform([original_text_combined, transcribed_text_combined])
    vectors = vectorizer.toarray()

    cosine_sim = cosine_similarity(vectors)
    return cosine_sim[0, 1]

def detailed_diff(original_text, transcribed_text):
    original_text_norm = normalize_text(original_text)
    transcribed_text_norm = normalize_text(transcribed_text)
    diff = ndiff(original_text_norm.split(), transcribed_text_norm.split())
    return '\n'.join(diff)

In [39]:
levenshtein_distance = calculate_levenshtein_distance(long_text, transcribed_text)
print(f"Levenshtein Distance: {levenshtein_distance}")

cosine_sim = calculate_cosine_similarity(long_text, transcribed_text)
print(f"Cosine Similarity: {cosine_sim:.2f}")

diff = detailed_diff(long_text, transcribed_text)
print(" ", "Detailed Differences:", diff, sep='\n')

Levenshtein Distance: 17
Cosine Similarity: 0.99
 
Detailed Differences:
  in
  a
  bustling
  town
- there
?     -

+ their
?    +

  lived
  a
  curious
  kitten
  named
  milo
  milo
  had
  bright
  green
  eyes
  and
  a
  fluffy
  tail
  that
  always
  seemed
  to
  be
  in
  motion
  he
  loved
  to
  explore
  every
  nook
  and
  cranny
  making
  new
  friends
  along
  the
  way
  one
  day
  while
  chasing
  a
  butterfly
  milo
  wandered
  farther
  than
  ever
  before
  and
  stumbled
  upon
  an
  old
  enchanted
  garden
  in
  the
  garden
  flowers
  sang
  melodies
  and
  trees
  whispered
  ancient
  secrets
  milos
  whiskers
  twitched
  with
  excitement
  as
  he
  ventured
  deeper
  he
  found
  a
  tiny
  golden
  door
  at
  the
  base
  of
  a
  giant
  oak
  tree
  unable
  to
  resist
  milo
  pushed
  it
  open
  and
  stepped
  inside
  to
  his
  amazement
  he
  found
  himself
  in
  a
  land
  of
  miniature
  creatures
  tiny
  fairies
- flitt

# Role Play

## Disription
* **Step 1: Creating and Sending the Card to the User** <br> Let's start by creating a card and sending it to the user.
* **Step 2: Let the user familiarize himself with his card**
* **Step 3: Starting a 5-Minute Timer for Role-Play** <br> Start a 5-minute timer and begin accepting a voice message from the user.
* **Loop:**
    * **Step 3.1: Receiving a Voice Message from the User** <br> Use the python-telegram-bot library to handle voice messages.
    * **Step 3.2: Transcribing the Voice Message with Whisper** <br> Integrate Whisper for processing voice messages.
    * **Step 3.3: Generating an Audio Response - Patient's Reply** <br> Use OpenAI GPT-4 to generate the patient's reply text.
    * **Step 3.4: Sending the Audio Response to the User** <br> Use Google Text-to-Speech to create an audio file and send the generated audio file to the user.

In [40]:
print('Now introduce yourself as a doctor and record your first voice message. You have 5 minutes.')

# 300 seconds timer

Now introduce yourself as a doctor and record your first voice message. You have 5 minutes.


We've received the first audio from a user

In [41]:
user_text = transcribe_with_whisper(whisper_model, path)
user_text

' Hello, my name is Pete. I am a doctor of this clinic. May I know your name and please tell me what is bothering you?'

Patient model content.

In [42]:
system_content = f"""
You play a patient in a role-playing game, you have a situation card: { {formatted_cards['patient']} }.
All you need to do is to FOLLOW the role play card.
Don't dump all the information at once!
You may be asked to tell what you came with, don't answer with sentences from the task, just give a short description (take it from your card).
Your task is to complete your tasks one by one, either until you are asked about something or until you are asked what is bothering you. 
Try not to go beyond what's on your card. If you are asked something not on your card, answer it briefly.
"""
print(system_content)

# Your name is AI patient.
# If they ask you your name - you can make it up, but regarding the doctor's questions about your condition, answer only what is on the notes.
# Do not move on to the next question until you have received an answer to the previous one. 


You play a patient in a role-playing game, you have a situation card: {"Patient Card:\n**SETTING**: Dermatology Clinic\n**PATIENT**: You are a 30-year-old woman who has been experiencing severe itching and redness on your skin for the past few weeks. You have noticed the symptoms mainly on your arms, legs, and back. The itching is disrupting your sleep and daily activities, and you have tried over-the-counter creams with no relief.\n**PATIENT TASKS**\n  - Describe the severity of the itching and redness on your skin, mentioning the areas affected.\n  - Express how the itching is impacting your sleep and daily life.\n  - Inform the doctor about the unsuccessful use of over-the-counter creams.\n  - Ask the doctor about the possible causes of your symptoms and if it could be a skin allergy.\n  - Acknowledge the doctor's recommendations and ask about the next steps.\n"}.
All you need to do is to FOLLOW the role play card.
Don't dump all the information at once!
You may be asked to tell wh

In [43]:
messages = [{"role": "system", "content": system_content}] # Append a system content

def chatbot_response(user_input):
    messages.append({"role": "user", "content": user_input}) # Append user's message
    
    completion = client.chat.completions.create(
        model=openai_model,
        messages=messages,
        max_tokens=200
    )
    
    output_text = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": output_text}) # Append assistant's message
    return output_text

In [45]:
print(formatted_cards['doctor'])

Doctor Card:
**SETTING**: Dermatology Clinic
**DOCTOR**: Your patient is a 30-year-old woman presenting with severe itching and redness on her arms, legs, and back. She reports disrupted sleep and daily activities despite trying over-the-counter creams.
**DOCTOR TASKS**
  - Inquire about the duration and progression of the symptoms.
  - Discuss potential triggers for the itching and redness (e.g., new products, environmental changes).
  - Explain the possible causes of the symptoms (e.g., contact dermatitis, eczema) and the importance of a proper diagnosis.
  - Recommend allergy testing or patch testing to identify potential allergens.
  - Provide treatment options such as prescription creams, lifestyle modifications, and follow-up care.



In [44]:
user_input = input("You: ")

while user_input.lower() != 'exit':
    response = chatbot_response(user_input)
    print(f"AI: {response}")
    
    user_input = input("You: ")

You:  exit
